# LSGAN_MNIST_Keras

https://github.com/eriklindernoren/Keras-GAN/blob/master/lsgan/lsgan.py

# 1. Load Modules

In [13]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import *
from keras.models import *
from keras.optimizers import *

import matplotlib.pyplot as plt
import sys
import numpy as np
import tensorflow as tf

# 2. Build Network

In [59]:
class LSGAN():
    
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows,self.img_cols, self.channels)
        self.latent_dim = 100
        
        optimizer = Adam(0.0002,0.5)
        
        # Build and Compile the discriminator
        self.discriminator = self.build_discriminator()
        # Loss = Mean Squared Error
        self.discriminator.compile(loss='mse',
                                  optimizer=optimizer,
                                  metrics=['accuracy'])
        
        # Build Generator
        self.generator = self.build_generator()
        
        # Make Noise
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)
        
        # For the combined model only train the generator
        self.discriminator.trainable = False
        
        # valid takes generated imgs as input and determines validity
        valid = self.discriminator(img)
        
        # The Combined model ( G + D)
        # Trains generator to fool discriminator
        self.combined = Model(z,valid)
        
        # Opitimize with MSE
        self.combined.compile(loss='mse',optimizer=optimizer)
        
    def build_generator(self):
        
        model = Sequential()
        
        model.add(Dense(256,input_dim=self.latent_dim))
        model.add(ReLU())
        model.add(BatchNormalization(momentum=0.8))
        
        model.add(Dense(512))
        model.add(ReLU())
        model.add(BatchNormalization(momentum=0.8))
        
        model.add(Dense(1024))
        model.add(ReLU())
        model.add(BatchNormalization(momentum=0.8))
        
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))   #784
        model.add(Reshape(self.img_shape))
        
        model.summary()
        
        noise = Input(shape=(self.latent_dim,))
        img = model(noise)
        
        return Model(noise,img)
    
    def build_discriminator(self):
        
        model = Sequential()
        
        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(0.2))
        
        model.add(Dense(256))
        model.add(LeakyReLU(0.2))
        model.add(Dense(1))   # no Softmax
        model.summary()
        
        img = Input(shape=self.img_shape)
        validity = model(img)
        
        return Model(img,validity)
    
    def train(self, epochs, batch_size=128,sample_interval=50):
        # Load MNIST
        
        (X_train,_),(_,_) = mnist.load_data()
        
        # Rescale
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train,axis=3) # 다 1열로 펼치기
        
        # Adversarial Ground-Truth ( real value )
        real = np.ones((batch_size,1))
        fake = np.zeros((batch_size,1))
        
        for epoch in range(epochs):
            
            """Train Discriminator"""
            
            # Select random batch of images
            idx = np.random.randint(0, X_train.shape[0],batch_size)  # 0 ~ X_train.shape[0]까지 batch_size 만큼
            imgs = X_train[idx]
            
            # Sample noise as generator input
            noise = np.random.normal(0,1,(batch_size,self.latent_dim))  # batch_size 행 / latent_dim 열
            
            # Generate a batch of new imgs
            gen_imgs = self.generator.predict(noise)
            
            # Train the Discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs,real)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs,fake)
            d_loss = 0.5 * np.add(d_loss_fake,d_loss_real)
            
            
            """Train Generator"""
            
            g_loss = self.combined.train_on_batch(noise,real)
            
            # Plot the progress
            print("Epoch : {0} / D_Loss : {1}, ACC : {2:.2f} / G_Loss : {3}".format(epoch,d_loss[0],100*d_loss[1],g_loss))
            
            # If at save interval => Save img samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)
                
    def sample_images(self,epoch):
        r,c = 5,5
        noise = np.random.normal(0,1,(r*c,self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale
        gen_imgs = gen_imgs * 0.5 + 0.5

        fig, axs = plt.subplots(r,c)
        cnt = 0

        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt,:,:,0],cmap='gray')
                axs[i,j].axis('off')
                cnt+=1
        fig.savefig("images/mnist_%d.png" % epoch)
        plt.close()    

# 3. Train the MNIST

In [60]:
if __name__ == '__main__':
    lsgan = LSGAN()
    lsgan.train(epochs=30000,batch_size=64, sample_interval=200)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_13 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_82 (Dense)             (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_31 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_83 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_32 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_84 (Dense)             (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

Epoch : 63 / D_Loss : 0.39710119366645813, ACC : 36.72 / G_Loss : 1.0852932929992676
Epoch : 64 / D_Loss : 0.31775766611099243, ACC : 50.00 / G_Loss : 1.138929009437561
Epoch : 65 / D_Loss : 0.18907782435417175, ACC : 73.44 / G_Loss : 0.9692761898040771
Epoch : 66 / D_Loss : 0.09923519939184189, ACC : 90.62 / G_Loss : 1.0372819900512695
Epoch : 67 / D_Loss : 0.05164875090122223, ACC : 97.66 / G_Loss : 1.1107170581817627
Epoch : 68 / D_Loss : 0.0822988897562027, ACC : 86.72 / G_Loss : 0.9042568802833557
Epoch : 69 / D_Loss : 0.1073434054851532, ACC : 89.06 / G_Loss : 1.0075749158859253
Epoch : 70 / D_Loss : 0.07640635967254639, ACC : 92.97 / G_Loss : 1.0619683265686035
Epoch : 71 / D_Loss : 0.056963831186294556, ACC : 92.97 / G_Loss : 1.0264363288879395
Epoch : 72 / D_Loss : 0.04847197234630585, ACC : 96.88 / G_Loss : 1.0342180728912354
Epoch : 73 / D_Loss : 0.06820496171712875, ACC : 93.75 / G_Loss : 1.2274476289749146
Epoch : 74 / D_Loss : 0.06770201772451401, ACC : 92.97 / G_Loss : 1

Epoch : 160 / D_Loss : 0.03802056983113289, ACC : 99.22 / G_Loss : 0.9647833704948425
Epoch : 161 / D_Loss : 0.06009156256914139, ACC : 95.31 / G_Loss : 0.8485886454582214
Epoch : 162 / D_Loss : 0.054926760494709015, ACC : 97.66 / G_Loss : 1.0273017883300781
Epoch : 163 / D_Loss : 0.07838727533817291, ACC : 96.88 / G_Loss : 0.8814997673034668
Epoch : 164 / D_Loss : 0.058837153017520905, ACC : 96.09 / G_Loss : 1.0834722518920898
Epoch : 165 / D_Loss : 0.05787111073732376, ACC : 96.09 / G_Loss : 1.0726490020751953
Epoch : 166 / D_Loss : 0.07473728805780411, ACC : 92.97 / G_Loss : 0.9957695007324219
Epoch : 167 / D_Loss : 0.1241576224565506, ACC : 80.47 / G_Loss : 1.1685819625854492
Epoch : 168 / D_Loss : 0.15351629257202148, ACC : 79.69 / G_Loss : 0.9502778053283691
Epoch : 169 / D_Loss : 0.1777953803539276, ACC : 77.34 / G_Loss : 1.0589898824691772
Epoch : 170 / D_Loss : 0.1483791172504425, ACC : 85.16 / G_Loss : 0.8365845680236816
Epoch : 171 / D_Loss : 0.09225142002105713, ACC : 92.97

Epoch : 260 / D_Loss : 0.05467647686600685, ACC : 96.09 / G_Loss : 0.9672776460647583
Epoch : 261 / D_Loss : 0.03557111322879791, ACC : 97.66 / G_Loss : 0.9212903380393982
Epoch : 262 / D_Loss : 0.06397028267383575, ACC : 95.31 / G_Loss : 0.7902691960334778
Epoch : 263 / D_Loss : 0.06639780104160309, ACC : 94.53 / G_Loss : 1.1333239078521729
Epoch : 264 / D_Loss : 0.18452832102775574, ACC : 74.22 / G_Loss : 0.6601482629776001
Epoch : 265 / D_Loss : 0.09538859874010086, ACC : 91.41 / G_Loss : 0.9654892086982727
Epoch : 266 / D_Loss : 0.17103618383407593, ACC : 82.03 / G_Loss : 0.8510565757751465
Epoch : 267 / D_Loss : 0.044145964086055756, ACC : 97.66 / G_Loss : 1.038185954093933
Epoch : 268 / D_Loss : 0.10422638058662415, ACC : 86.72 / G_Loss : 0.8992401361465454
Epoch : 269 / D_Loss : 0.040605779737234116, ACC : 95.31 / G_Loss : 0.8913311958312988
Epoch : 270 / D_Loss : 0.06393631547689438, ACC : 91.41 / G_Loss : 0.8593186140060425
Epoch : 271 / D_Loss : 0.08790116012096405, ACC : 84.

Epoch : 357 / D_Loss : 0.14087261259555817, ACC : 80.47 / G_Loss : 0.8206331133842468
Epoch : 358 / D_Loss : 0.11312402039766312, ACC : 86.72 / G_Loss : 0.7536754608154297
Epoch : 359 / D_Loss : 0.0719929039478302, ACC : 90.62 / G_Loss : 0.8967832326889038
Epoch : 360 / D_Loss : 0.14033576846122742, ACC : 82.03 / G_Loss : 0.7140394449234009
Epoch : 361 / D_Loss : 0.07249864190816879, ACC : 89.84 / G_Loss : 0.909273624420166
Epoch : 362 / D_Loss : 0.15236511826515198, ACC : 78.12 / G_Loss : 0.7786455154418945
Epoch : 363 / D_Loss : 0.1344485580921173, ACC : 81.25 / G_Loss : 0.9742299318313599
Epoch : 364 / D_Loss : 0.1499716341495514, ACC : 82.03 / G_Loss : 0.6882973909378052
Epoch : 365 / D_Loss : 0.08275977522134781, ACC : 90.62 / G_Loss : 0.9180949926376343
Epoch : 366 / D_Loss : 0.09980988502502441, ACC : 86.72 / G_Loss : 0.9082750082015991
Epoch : 367 / D_Loss : 0.16645176708698273, ACC : 77.34 / G_Loss : 0.7847294807434082
Epoch : 368 / D_Loss : 0.07972365617752075, ACC : 89.06 / 

...

Epoch : 29577 / D_Loss : 0.24147161841392517, ACC : 51.56 / G_Loss : 0.31686288118362427
Epoch : 29578 / D_Loss : 0.23393960297107697, ACC : 63.28 / G_Loss : 0.3240372836589813
Epoch : 29579 / D_Loss : 0.23273780941963196, ACC : 61.72 / G_Loss : 0.32339462637901306
Epoch : 29580 / D_Loss : 0.24839212000370026, ACC : 53.91 / G_Loss : 0.31190603971481323
Epoch : 29581 / D_Loss : 0.24803036451339722, ACC : 53.91 / G_Loss : 0.32074135541915894
Epoch : 29582 / D_Loss : 0.2519606947898865, ACC : 52.34 / G_Loss : 0.31926894187927246
Epoch : 29583 / D_Loss : 0.23788997530937195, ACC : 54.69 / G_Loss : 0.3125333786010742
Epoch : 29584 / D_Loss : 0.24413053691387177, ACC : 52.34 / G_Loss : 0.32272952795028687
Epoch : 29585 / D_Loss : 0.23887555301189423, ACC : 53.91 / G_Loss : 0.32160139083862305
Epoch : 29586 / D_Loss : 0.24095341563224792, ACC : 57.81 / G_Loss : 0.3407682776451111
Epoch : 29587 / D_Loss : 0.2322608083486557, ACC : 58.59 / G_Loss : 0.3202170431613922
Epoch : 29588 / D_Loss : 0.

Epoch : 29673 / D_Loss : 0.23764431476593018, ACC : 60.94 / G_Loss : 0.31174176931381226
Epoch : 29674 / D_Loss : 0.25920993089675903, ACC : 44.53 / G_Loss : 0.29677248001098633
Epoch : 29675 / D_Loss : 0.2511741518974304, ACC : 53.91 / G_Loss : 0.30206406116485596
Epoch : 29676 / D_Loss : 0.23853640258312225, ACC : 57.81 / G_Loss : 0.3158091604709625
Epoch : 29677 / D_Loss : 0.25101229548454285, ACC : 51.56 / G_Loss : 0.314028263092041
Epoch : 29678 / D_Loss : 0.23798927664756775, ACC : 60.16 / G_Loss : 0.317185640335083
Epoch : 29679 / D_Loss : 0.23491550981998444, ACC : 57.03 / G_Loss : 0.33001816272735596
Epoch : 29680 / D_Loss : 0.247210294008255, ACC : 53.12 / G_Loss : 0.3194121718406677
Epoch : 29681 / D_Loss : 0.2368101179599762, ACC : 60.16 / G_Loss : 0.3334904909133911
Epoch : 29682 / D_Loss : 0.25004199147224426, ACC : 57.03 / G_Loss : 0.32146936655044556
Epoch : 29683 / D_Loss : 0.24317069351673126, ACC : 53.12 / G_Loss : 0.3074387311935425
Epoch : 29684 / D_Loss : 0.248391

Epoch : 29769 / D_Loss : 0.24841776490211487, ACC : 55.47 / G_Loss : 0.3313705623149872
Epoch : 29770 / D_Loss : 0.25008174777030945, ACC : 58.59 / G_Loss : 0.30365562438964844
Epoch : 29771 / D_Loss : 0.225447416305542, ACC : 65.62 / G_Loss : 0.3032763600349426
Epoch : 29772 / D_Loss : 0.2542850375175476, ACC : 49.22 / G_Loss : 0.32576608657836914
Epoch : 29773 / D_Loss : 0.24608562886714935, ACC : 56.25 / G_Loss : 0.3236496150493622
Epoch : 29774 / D_Loss : 0.2457309067249298, ACC : 51.56 / G_Loss : 0.29468488693237305
Epoch : 29775 / D_Loss : 0.25586920976638794, ACC : 46.88 / G_Loss : 0.30927789211273193
Epoch : 29776 / D_Loss : 0.23901718854904175, ACC : 57.03 / G_Loss : 0.33284032344818115
Epoch : 29777 / D_Loss : 0.23341676592826843, ACC : 59.38 / G_Loss : 0.3252091705799103
Epoch : 29778 / D_Loss : 0.2391161024570465, ACC : 56.25 / G_Loss : 0.3099479377269745
Epoch : 29779 / D_Loss : 0.24347084760665894, ACC : 53.12 / G_Loss : 0.30486929416656494
Epoch : 29780 / D_Loss : 0.2461

Epoch : 29863 / D_Loss : 0.24043475091457367, ACC : 57.03 / G_Loss : 0.31627917289733887
Epoch : 29864 / D_Loss : 0.237228661775589, ACC : 60.16 / G_Loss : 0.31566935777664185
Epoch : 29865 / D_Loss : 0.23263435065746307, ACC : 63.28 / G_Loss : 0.3179782032966614
Epoch : 29866 / D_Loss : 0.24191543459892273, ACC : 55.47 / G_Loss : 0.29253244400024414
Epoch : 29867 / D_Loss : 0.2549920082092285, ACC : 50.78 / G_Loss : 0.308445543050766
Epoch : 29868 / D_Loss : 0.2434328943490982, ACC : 60.16 / G_Loss : 0.306560754776001
Epoch : 29869 / D_Loss : 0.2380843460559845, ACC : 61.72 / G_Loss : 0.3191097676753998
Epoch : 29870 / D_Loss : 0.24387486279010773, ACC : 56.25 / G_Loss : 0.3060982823371887
Epoch : 29871 / D_Loss : 0.2476736307144165, ACC : 51.56 / G_Loss : 0.2964930534362793
Epoch : 29872 / D_Loss : 0.23486343026161194, ACC : 57.03 / G_Loss : 0.3072297275066376
Epoch : 29873 / D_Loss : 0.24871067702770233, ACC : 54.69 / G_Loss : 0.32673949003219604
Epoch : 29874 / D_Loss : 0.231507495

Epoch : 29959 / D_Loss : 0.239088237285614, ACC : 55.47 / G_Loss : 0.32955068349838257
Epoch : 29960 / D_Loss : 0.2414492666721344, ACC : 58.59 / G_Loss : 0.32110679149627686
Epoch : 29961 / D_Loss : 0.2582188546657562, ACC : 51.56 / G_Loss : 0.28604474663734436
Epoch : 29962 / D_Loss : 0.244985431432724, ACC : 52.34 / G_Loss : 0.30663976073265076
Epoch : 29963 / D_Loss : 0.2433268427848816, ACC : 55.47 / G_Loss : 0.318789541721344
Epoch : 29964 / D_Loss : 0.23589354753494263, ACC : 61.72 / G_Loss : 0.32761093974113464
Epoch : 29965 / D_Loss : 0.2543945908546448, ACC : 49.22 / G_Loss : 0.305075466632843
Epoch : 29966 / D_Loss : 0.24597463011741638, ACC : 54.69 / G_Loss : 0.30584755539894104
Epoch : 29967 / D_Loss : 0.24588894844055176, ACC : 50.00 / G_Loss : 0.3166544437408447
Epoch : 29968 / D_Loss : 0.260809063911438, ACC : 47.66 / G_Loss : 0.32058262825012207
Epoch : 29969 / D_Loss : 0.24760352075099945, ACC : 57.03 / G_Loss : 0.31944459676742554
Epoch : 29970 / D_Loss : 0.262423902

# 4. Make GIF

In [153]:
# 파일이름의 숫자 5자리로 만들어서 정렬하기
import re
path = os.getcwd()
filename = glob.glob(path + '/images/mnist_*.png')
for i in filename:
    a = i.split('/')[-1]
    leng = re.findall('\d+',a)[0]
    leng = "0"*(5-len(leng))+leng
    os.rename(i,leng+".png",path+'/images')

In [160]:
filename_new = glob.glob(path + '/images/*.png')
filename_new = sorted(filename_new)
filename_new[:5]
# 순서대로 정렬된 것을 볼 수 있다. 

['/Users/charming/Python/0_Paper_Review/09. LSGAN/images/00000.png',
 '/Users/charming/Python/0_Paper_Review/09. LSGAN/images/00200.png',
 '/Users/charming/Python/0_Paper_Review/09. LSGAN/images/00400.png',
 '/Users/charming/Python/0_Paper_Review/09. LSGAN/images/00600.png',
 '/Users/charming/Python/0_Paper_Review/09. LSGAN/images/00800.png']

In [162]:
generated_image_array = [imageio.imread(generated_image) for generated_image in filename_new]
imageio.mimsave('LSGAN_MNIST.gif', generated_image_array, fps=15)